In [1]:
import os
import time

from PIL import Image

def fullpage_screenshot(driver, file):

        print("Starting chrome full page screenshot workaround ...")

        total_width = driver.execute_script("return document.body.offsetWidth")
        total_height = driver.execute_script("return document.body.parentNode.scrollHeight")
        viewport_width = driver.execute_script("return document.body.clientWidth")
        viewport_height = driver.execute_script("return window.innerHeight")
        print("Total: ({0}, {1}), Viewport: ({2},{3})".format(total_width, total_height,viewport_width,viewport_height))
        rectangles = []

        i = 0
        while i < total_height:
            ii = 0
            top_height = i + viewport_height

            if top_height > total_height:
                top_height = total_height

            while ii < total_width:
                top_width = ii + viewport_width

                if top_width > total_width:
                    top_width = total_width

                print("Appending rectangle ({0},{1},{2},{3})".format(ii, i, top_width, top_height))
                rectangles.append((ii, i, top_width,top_height))

                ii = ii + viewport_width

            i = i + viewport_height

        stitched_image = Image.new('RGB', (total_width, total_height))
        previous = None
        part = 0

        for rectangle in rectangles:
            if not previous is None:
                driver.execute_script("window.scrollTo({0}, {1})".format(rectangle[0], rectangle[1]))
                print("Scrolled To ({0},{1})".format(rectangle[0], rectangle[1]))
                time.sleep(0.2)

            file_name = "part_{0}.png".format(part)
            print("Capturing {0} ...".format(file_name))

            driver.get_screenshot_as_file(file_name)
            screenshot = Image.open(file_name)

            if rectangle[1] + viewport_height > total_height:
                offset = (rectangle[0], total_height - viewport_height)
            else:
                offset = (rectangle[0], rectangle[1])

            print("Adding to stitched image with offset ({0}, {1})".format(offset[0],offset[1]))
            stitched_image.paste(screenshot, offset)

            del screenshot
            os.remove(file_name)
            part = part + 1
            previous = rectangle

        stitched_image.save(file)
        print("Finishing chrome full page screenshot workaround...")
        return True

In [2]:
"""
This script uses a simplified version of the one here:
https://snipt.net/restrada/python-selenium-workaround-for-full-page-screenshot-using-chromedriver-2x/

It contains the *crucial* correction added in the comments by Jason Coutu.
"""
import sys

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

import unittest

class Test(unittest.TestCase):

    def setUp(self):
        self.driver = webdriver.Chrome(ChromeDriverManager().install())

    def tearDown(self):
        self.driver.quit()

    def test_fullpage_screenshot(self):
        ''' Generate document-height screenshot '''
        url = "https://www.tiendeo.us/new-york-city-NY/cvs-pharmacy"
        self.driver.get(url)
        self.driver.implicitly_wait(5)

        fullpage_screenshot(self.driver, "test.png")

unittest.main(argv=[sys.argv[0]])



Checking for win32 chromedriver:2.46 in cache
Driver found in C:\Users\Orwell\.wdm\chromedriver\2.46\win32/chromedriver.exe


c:\users\orwell\documents\gitprojects\seleniumpybcn\env\lib\site-packages\ipykernel_launcher.py:17: ResourceWarning: unclosed file <_io.TextIOWrapper name='C:\\Users\\Orwell\\.wdm\\chromedriver\\2.46\\win32\\chromedriver.exe' mode='r' encoding='cp65001'>


KeyboardInterrupt: 

In [ ]:
#chrome
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager #install the driver automatically
# https://allselenium.info/browser-drivers-with-python-webdrivermanager/

browser = webdriver.Chrome(ChromeDriverManager().install())
browser.set_window_size(250, 700)
#browser.set_window_position(200, 200)
browser.get("https://www.tiendeo.us/new-york-city-NY/cvs-pharmacy")

In [ ]:
#firefox
import time
from webdriver_manager.firefox import GeckoDriverManager
from selenium import webdriver

browser = webdriver.Firefox(executable_path=GeckoDriverManager().install())
browser.set_window_size(360, 640)
#browser.set_window_position(200, 200)
browser.get("https://www.tiendeo.us/new-york-city-NY/cvs-pharmacy")
#browser.implicitly_wait(5)

time.sleep(2)
browser.save_screenshot("screenshot.png")







browser.quit()

In [ ]:
import os
import sys
import time
from PIL import ImageOps, Image

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager


# https://allselenium.info/browser-drivers-with-python-webdrivermanager/

browser = webdriver.Chrome(ChromeDriverManager().install())
browser.set_window_size(250, 600)

browser.get("https://www.tiendeo.us/new-york-city-NY/cvs-pharmacy")

total_width = browser.execute_script("return document.body.offsetWidth")
total_height = browser.execute_script("return document.body.parentNode.scrollHeight")
viewport_width = browser.execute_script("return document.body.clientWidth")
viewport_height = browser.execute_script("return window.innerHeight")
pixel_ratio = browser.execute_script("return window.devicePixelRatio")
header_height = browser.execute_script("return document.getElementsByClassName('s·header')[0].offsetHeight")

print("Total: ({0}, {1}), Viewport: ({2},{3}), PixelRatio: {4}".format(total_width, total_height,viewport_width,viewport_height,pixel_ratio))

time.sleep(5)

browser.get_screenshot_as_file('part_0.png')

browser.execute_script("document.getElementsByClassName('s·header')[0].style.boxShadow = 'none'")  # hide shadow 

height_without_header = viewport_height - header_height

for mid_slice in range(1, int(total_height/height_without_header)+1):
    browser.execute_script('window.scrollBy(0, {})'.format(height_without_header))
    time.sleep(0.2)
    browser.get_screenshot_as_file('part_{}.png'.format(mid_slice))
    border = (0, header_height * pixel_ratio, 0, 0) # left, up, right, bottom
    img = Image.open('part_{}.png'.format(mid_slice))
    img = ImageOps.crop(img, border)
    img.save('part_{}.png'.format(mid_slice))

browser.execute_script('window.scrollBy(0, 469-104)')
time.sleep(0.2)
browser.get_screenshot_as_file('part_2.png')
border = (0, 104 * 1.5, 0, 0) # left, up, right, bottom
img = Image.open("part_2.png")
img = ImageOps.crop(img, border)
img.save('part_2.png')

browser.execute_script('window.scrollBy(0, 469-104)')
time.sleep(0.2)
browser.get_screenshot_as_file('part_3.png')
border = (0, 104 * 1.5, 0, 0) # left, up, right, bottom
img = Image.open("part_3.png")
img = ImageOps.crop(img, border)
img.save('part_3.png')

browser.execute_script('window.scrollBy(0, 469-104)')
time.sleep(0.2)
browser.get_screenshot_as_file('part_4.png')
border = (0, 104 * 1.5, 0, 0) # left, up, right, bottom
img = Image.open("part_4.png")
img = ImageOps.crop(img, border)
img.save('part_4.png')

browser.execute_script('window.scrollBy(0, 469-104)')
time.sleep(0.2)
browser.get_screenshot_as_file('part_5.png')
border = (0, 104 * 1.5, 0, 0) # left, up, right, bottom
img = Image.open("part_5.png")
img = ImageOps.crop(img, border)
img.save('part_5.png')

browser.execute_script('window.scrollBy(0, 469-104)')
time.sleep(0.2)
browser.get_screenshot_as_file('part_6.png')
border = (0, 104 * 1.5, 0, 0) # left, up, right, bottom
img = Image.open("part_6.png")
img = ImageOps.crop(img, border)
img.save('part_6.png')

browser.execute_script('window.scrollBy(0, 469-104)')
time.sleep(0.2)
browser.get_screenshot_as_file('part_7.png')
border = (0, 104 * 1.5, 0, 0) # left, up, right, bottom
img = Image.open("part_7.png")
img = ImageOps.crop(img, border)
img.save('part_7.png')

browser.execute_script('window.scrollBy(0, 469-104)')
time.sleep(0.2)
browser.get_screenshot_as_file('part_8.png')
border = (0, 104 * 1.5, 0, 0) # left, up, right, bottom
img = Image.open("part_8.png")
img = ImageOps.crop(img, border)
img.save('part_8.png')

browser.execute_script('window.scrollBy(0, 469-104)')
time.sleep(0.2)
browser.get_screenshot_as_file('part_9.png')
border = (0, 104 * 1.5, 0, 0) # left, up, right, bottom
img = Image.open("part_9.png")
img = ImageOps.crop(img, border)
img.save('part_9.png')

browser.execute_script('window.scrollBy(0, 469-104)')
time.sleep(0.2)
browser.get_screenshot_as_file('part_10.png')
border = (0, 104*1.5, 0, 0) # left, up, right, bottom
img = Image.open("part_10.png")
img = ImageOps.crop(img, border)
img.save('part_10.png')

remaining_height = (total_height-viewport_height) % (viewport_height - 104)

browser.execute_script('window.scrollBy(0, 469-104)')
time.sleep(0.2)
browser.get_screenshot_as_file('part_11.png')
border = (0, (viewport_height - remaining_height)  * 1.5, 0, 0) # left, up, right, bottom
img = Image.open("part_11.png")
img = ImageOps.crop(img, border)
img.save('part_11.png')

In [ ]:
for a in range(1, int(total_height/(viewport_height-104))+1):
    print (a)

In [ ]:
Image.open("part_11.png").size[1] - 133 * 1.5

In [ ]:
images = [Image.open(image_name) for image_name in ['part_0.png', 'part_1.png', 'part_2.png', 'part_3.png', 'part_4.png', 'part_5.png', 'part_6.png', 'part_7.png', 'part_8.png', 'part_9.png', 'part_10.png', 'part_11.png']]

widths, heights = zip(*(i.size for i in images))

total_width = max(widths)
max_height = sum(heights)

new_im = Image.new('RGB', (total_width, max_height))

y_offset = 0

for im in images:
    new_im.paste(im, (0, y_offset))
    y_offset += im.size[1]

new_im.save('full_screenshot.png')

In [ ]:
rectangles = []
i = 0

while i < total_height:
    ii = 0
    top_height = i + viewport_height - 104

    if top_height > total_height:
        top_height = total_height

    while ii < total_width:
        top_width = ii + viewport_width

        if top_width > total_width:
            top_width = total_width

        print("Appending rectangle ({0},{1},{2},{3})".format(ii, i, top_width, top_height))
        rectangles.append((ii, i, top_width,top_height))

        ii = ii + viewport_width

    i = i + viewport_height - 140

stitched_image = Image.new('RGB', (total_width, total_height))
previous = None
part = 0

for rectangle in rectangles:
    if not previous is None:
        browser.execute_script("window.scrollTo({0}, {1})".format(rectangle[0], rectangle[1]))
        print("Scrolled To ({0},{1})".format(rectangle[0], rectangle[1]))
        time.sleep(1)

    file_name = "part_{0}.png".format(part)
    print("Capturing {0} ...".format(file_name))

    browser.get_screenshot_as_file(file_name)
    screenshot = Image.open(file_name)

    if rectangle[1] + viewport_height > total_height:
        offset = (rectangle[0], total_height - viewport_height)
    else:
        offset = (rectangle[0], rectangle[1])

    print("Adding to stitched image with offset ({0}, {1})".format(offset[0],offset[1]))
    stitched_image.paste(screenshot, offset)

    #del screenshot
    #os.remove(file_name)
    part = part + 1
    previous = rectangle

stitched_image.save('final.png')
print("Finishing chrome full page screenshot workaround...")
browser.quit()

In [ ]:
browser.quit()

In [ ]:

import os
import time

from PIL import Image

i = 0
while i < total_height:
    ii = 0
    top_height = i + viewport_height

    if top_height > total_height:
        top_height = total_height

    while ii < total_width:
        top_width = ii + viewport_width

        if top_width > total_width:
            top_width = total_width

        print("Appending rectangle ({0},{1},{2},{3})".format(ii, i, top_width, top_height))
        rectangles.append((ii, i, top_width,top_height))

        ii = ii + viewport_width

    i = i + viewport_height

stitched_image = Image.new('RGB', (total_width, total_height))
previous = None
part = 0

In [ ]:
rectangles

In [ ]:
from PIL import ImageOps

border = (0, 104 * 1.5, 0, 0) # left, up, right, bottom

img = Image.open("screenshot.png")

ImageOps.crop(img, border)

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager #install the driver automatically

from PIL import Image
from io import BytesIO

fox = webdriver.Chrome(ChromeDriverManager().install())
fox.get('http://stackoverflow.com/')

aspect_ratio = fox.execute_script("return window.devicePixelRatio")

In [ ]:
# now that we have the preliminary stuff out of the way time to get that image :D
element = fox.find_element_by_id('search') # find part of the page you want image of
location = element.location
size = element.size
png = fox.get_screenshot_as_png() # saves screenshot of entire page
fox.quit()

im = Image.open(BytesIO(png)) # uses PIL library to open image in memory

left = location['x']*aspect_ratio
top = location['y']*aspect_ratio
right = location['x']*aspect_ratio + size['width']*aspect_ratio
bottom = location['y']*aspect_ratio + size['height']*aspect_ratio


im = im.crop((left, top, right, bottom)) # defines crop points
im.save('screenshot.png') # saves new cropped image

In [ ]:
element.save_screenshot('header.png')

In [ ]:
import sys
from PIL import Image

images = [Image.open(image_name) for image_name in ['part_0.png', 'part_1.png', 'part_2.png']]

widths, heights = zip(*(i.size for i in images))

total_width = max(widths)
max_height = sum(heights)

new_im = Image.new('RGB', (total_width, max_height))

y_offset = 0

#images = map(Image.open, ['part_0.png', 'part_1.png', 'part_2.png'])

for im in images:
    print ('a')
    new_im.paste(im, (0, y_offset))
    y_offset += im.size[1]

new_im#.save('test.jpg')

In [ ]:
images = map(Image.open, ['part_0.png', 'part_1.png', 'part_2.png'])
widths, heights = zip(*(i.size for i in images))

total_width = max(widths)
max_height = sum(heights)

new_im = Image.new('RGB', (total_width, max_height))

y_offset = 0
images = map(Image.open, ['part_0.png', 'part_1.png', 'part_2.png'])

In [ ]:
for im in images:
    print ('a')
    #new_im.paste(im, (0, y_offset))
    #y_offset += im.size[1]
    

In [ ]:
im.filename

In [ ]:
new_im.paste(image, (0, 20))

In [ ]:
new_im

In [ ]:
new_im